本教程提供了有关如何将CSV数据从文件加载到的示例tf.data.Dataset。

本教程中使用的数据取自《泰坦尼克号》乘客列表。该模型将根据年龄，性别，机票等级以及该人是否独自旅行等特征来预测乘客幸存的可能性。

## 1.下载数据

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 11us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

您可以使用pandas加载它，并将NumPy数组传递给TensorFlow。如果您需要扩展到一大堆文件，或者需要一个与TensorFlow和tf.data集成的加载器，请使用以下tf.data.experimental.make_csv_dataset函数：

您需要明确标识的唯一一列是具有模型要预测的值的列。

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

## 2.使用tf的API直接加载训练数据，并创建dataset

现在，从文件中读取CSV数据并创建一个数据集。

In [5]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        # 文件路径
        file_path,
        # 批量数
        batch_size=5, # Artificially small to make examples easier to show.
        # y标签
        label_name=LABEL_COLUMN,
        na_value="?",
        # 读取1个epoch
        num_epochs=1,
        ignore_errors=True, 
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [6]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

数据集中的每个项目都是一个批处理，表示为（许多示例，许多标签）的元组。示例中的数据以基于列的张量（而不是基于行的张量）进行组织，每个张量具有与批量大小一样多的元素（在这种情况下为5）。

自己看一下可能会有所帮助。

In [7]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [38. 25. 28. 28. 24.]
n_siblings_spouses  : [0 1 0 8 0]
parch               : [1 0 0 2 0]
fare                : [153.462  91.079   7.896  69.55    7.896]
class               : [b'First' b'First' b'Third' b'Third' b'Third']
deck                : [b'C' b'B' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'n' b'y']


如您所见，CSV中的列已命名。数据集构造函数将自动选择这些名称。如果您使用的文件的第一行不包含列名，则将它们以字符串列表的形式传递给column_names函数中的make_csv_dataset参数。

In [8]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'female' b'female']
age                 : [28. 28. 28. 47. 19.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [ 7.733  7.896  7.75  14.5   30.   ]
class               : [b'Third' b'Third' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Queenstown' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


本示例将使用所有可用的列。如果您需要从数据集中省略一些列，请创建仅计划使用的列的列表，然后将其传递到select_columns构造函数的（可选）参数中。

In [9]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [35. 24. 27. 39. 40.]
n_siblings_spouses  : [0 0 0 0 1]
class               : [b'First' b'Second' b'Third' b'Second' b'Second']
deck                : [b'C' b'F' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'y' b'y' b'n']


## 3、连续数字数据预处理
CSV文件可以包含多种数据类型。通常，您需要先将这些混合类型转换为固定长度的向量，然后再将数据输入模型。

TensorFlow有一个用于描述常见输入转换的内置系统：tf.feature_column，请参阅本教程以获取详细信息。

您可以使用任何喜欢的工具（例如nltk或sklearn）对数据进行预处理，然后将处理后的输出传递给TensorFlow。

在模型内部进行预处理的主要优点是，在导出模型时，它包括预处理。这样，您可以将原始数据直接传递到模型。

连续数据

如果您的数据已经采用适当的数字格式，则可以将数据打包到向量中，然后再传递给模型：

In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [24. 43. 29. 28. 30.]
n_siblings_spouses  : [0. 0. 0. 3. 0.]
parch               : [0. 0. 0. 1. 0.]
fare                : [ 7.896  8.05   9.5   25.467  7.896]


In [41]:
example_batch, labels_batch = next(iter(temp_dataset)) 

这是一个将所有列打包在一起的简单函数：

In [12]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

### 转一下数据

将此应用于数据集的每个元素：

In [13]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 25.      1.      0.      7.925]
 [ 49.      1.      0.     89.104]
 [ 38.      1.      5.     31.388]
 [ 30.      0.      0.    106.425]
 [ 50.      0.      0.     13.   ]]

[0 1 1 1 0]


如果您有混合数据类型，则可能需要将这些简单数字字段分开。该tf.feature_columnapi可以处理它们，但这会产生一些开销，除非真正必要，否则应避免使用。切换回混合数据集：

In [14]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [29. 28. 19. 28. 28.]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 0 0 0]
fare                : [9.483 0.    7.854 7.896 6.858]
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [15]:
example_batch, labels_batch = next(iter(temp_dataset)) 

### 获取指定列的数据，使用内置的会产生额外开销，因此，可以定义预处理器，来处理

因此，定义一个更通用的预处理器，该预处理器选择一系列数字特征并将其打包到单个列中：

In [18]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_freatures = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [19]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [20]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'Third' b'First' b'Second' b'Third' b'First']
deck                : [b'unknown' b'D' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'n' b'y' b'n']
numeric             : [[ 34.      0.      0.      8.05 ]
 [ 48.      1.      0.     76.729]
 [ 33.      0.      2.     26.   ]
 [ 51.      0.      0.      7.054]
 [ 24.      0.      1.    247.521]]


In [21]:
example_batch, labels_batch = next(iter(packed_train_data)) 

数据归一化

连续数据应始终标准化。

In [22]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [23]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [24]:
def normalize_numeric_data(data, mean, std):
    # Center the data
    return (data-mean)/std

现在创建一个数字列。该tf.feature_columns.numeric_columnAPI接受一个normalizer_fn参数，该参数将在每个批处理运行。

使用将MEAN和和绑定STD到归一化器fn functools.partial。

In [25]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002058F35E598>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [26]:
example_batch['numeric']

<tf.Tensor: id=550, shape=(5, 4), dtype=float32, numpy=
array([[ 28.   ,   8.   ,   2.   ,  69.55 ],
       [ 28.   ,   1.   ,   0.   ,  16.1  ],
       [ 30.   ,   0.   ,   0.   ,   7.225],
       [ 21.   ,   0.   ,   0.   ,   7.925],
       [ 28.   ,   1.   ,   0.   , 146.521]], dtype=float32)>

In [27]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  6.476,  2.043,  0.644],
       [-0.13 ,  0.395, -0.479, -0.335],
       [ 0.029, -0.474, -0.479, -0.497],
       [-0.69 , -0.474, -0.479, -0.485],
       [-0.13 ,  0.395, -0.479,  2.054]], dtype=float32)

## 4、类别数据预处理

这里使用的基于均值的归一化要求提前知道每一列的均值。

分类数据

CSV数据中的某些列是分类列。也就是说，内容应该是一组有限的选项之一。

使用tf.feature_columnAPI tf.feature_column.indicator_column为每个分类列创建一个。

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


稍后在构建模型时，这将成为数据处理输入的一部分。

组合预处理层

添加两个要素列集合，并将它们传递给，tf.keras.layers.DenseFeatures以创建一个输入层，该层将提取并预处理这两种输入类型：

In [32]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [33]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13   6.476
  2.043  0.644  0.     1.   ]


## 五、训练模型

建立模型

建立一个tf.keras.Sequential，以开头preprocessing_layer。

In [34]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

训练，评估和预测

现在可以实例化和训练模型。

In [35]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [36]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 16ms/step - loss: 0.5042 - accuracy: 0.7671
Epoch 2/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4329 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4031 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3870 - accuracy: 0.8246
Epoch 5/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3899 - accuracy: 0.8405
Epoch 6/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3831 - accuracy: 0.8246
Epoch 7/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3708 - accuracy: 0.8469
Epoch 8/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3666 - accuracy: 0.8373
Epoch 9/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3569 - accuracy: 0.8469
Epoch 10/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3571 - accuracy: 0.845

训练模型后，您可以在模型上检查其准确性test_data。

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 11ms/step - loss: 0.4363 - accuracy: 0.8523


Test Loss 0.43626573386619677, Test Accuracy 0.8522727489471436


使用tf.keras.Model.predict上一批或分批的数据集推断的标签。

In [38]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 52.06%  | Actual outcome:  DIED
Predicted survival: 48.60%  | Actual outcome:  DIED
Predicted survival: 13.70%  | Actual outcome:  SURVIVED
Predicted survival: 6.39%  | Actual outcome:  SURVIVED
Predicted survival: 87.34%  | Actual outcome:  SURVIVED
